# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read school file and check columns
school_df = pd.read_csv(school_data_to_load) 
school_df.columns

#Rename columns
school = school_df.rename(columns={'school_name' : 'School_name','type' : 'Type','size' : 'Size','budget' : 'Budget'})
school.head()



,School ID,School_name,Type,Size,Budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Read student file and check columns
student_df = pd.read_csv(student_data_to_load)
student_df.columns

# Rename columns 
student = student_df.rename(columns={'student_name' : 'Student_name','gender' : 'Gender','grade' : 'Grade',
                           'school_name' : 'School_name','reading_score' : 'Reading_Score', 'math_score' : 'Math_Score'})

student.head()



,Student ID,Student_name,Gender,Grade,School_name,Reading_Score,Math_Score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [3]:
#merging  student with school file
MergedData = pd.merge(student, school, on=["School_name"])
MergedData.head()

,Student ID,Student_name,Gender,Grade,School_name,Reading_Score,Math_Score,School ID,Type,Size,Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
# Find  total schools, students and budget
TotalSchools = MergedData["School_name"].nunique()
TotalStudents = MergedData["Student ID"].nunique()
TotalBudget = school["Budget"].sum()

# Find average Math and Reading score
AvgMath = MergedData["Math_Score"].mean()
AvgReading = MergedData["Reading_Score"].mean()

#Total count of passing Math, reading and overall passing count
PassMath = MergedData[MergedData["Math_Score"] > 70].count()["School_name"]
PassReading = MergedData[MergedData["Reading_Score"] > 70].count()["School_name"]
TotalPassCount = MergedData[(MergedData["Math_Score"] > 70) & (MergedData["Reading_Score"] > 70)].count()["School_name"]

# Percentage 
PercentPassMath = (PassMath / TotalStudents) * 100
PercentPassingReading = (PassReading / TotalStudents) * 100
PercentPassingTotal = (TotalPassCount / TotalStudents) * 100

# District Summary Table
DistrictSummary = pd.DataFrame({"Total Schools":[TotalSchools], "Total Students":[TotalStudents],
                                "Total Budget":[TotalBudget], "Average Math Score":[AvgMath],
                                 "Average Reading Score":[AvgReading], "% Passing Math":PercentPassMath,
                                 "% Passing Reading":PercentPassingReading, "Overall Passing Rate": PercentPassingTotal})

DistrictSummary = DistrictSummary[["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                                 "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]

DistrictSummary




,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,72.392137,82.971662,60.801634


In [5]:

# Determine the school type
SchoolTypes = school.set_index(["School_name"])["Type"]

# Find total student count per school
SchoolCounts = MergedData["School_name"].value_counts()

# Budget
SchoolBudget = MergedData.groupby(["School_name"]).mean()["Budget"]
StudentBudget = SchoolBudget/ SchoolCounts

# Average math and reading scores
AvgMath= MergedData.groupby(["School_name"]).mean()["Math_Score"]
AvgReading = MergedData.groupby(["School_name"]).mean()["Reading_Score"]

# Schools passing Math and Reading
SchoolPassMath =  MergedData[MergedData["Math_Score"] > 70].groupby("School_name").count()["Student_name"]
SchoolPassReading =  MergedData[MergedData["Reading_Score"] > 70].groupby("School_name").count()["Student_name"]

# Calculate percent 
PercentPassMath = SchoolPassMath / SchoolCounts * 100
PercentPassReading = SchoolPassReading / SchoolCounts * 100
TotakPass = (PercentPassMath + PercentPassReading) / 2

# Convert to data frame
SchoolSummary = pd.DataFrame({"School Type":SchoolTypes, "Total Students":SchoolCounts,
                               "Total School Budget":SchoolBudget, "Per Student Budget":StudentBudget,
                                "Average Math Score":AvgMath, "Average Reading Score":AvgReading,
                                 "% Passing Math":PercentPassMath, "% Passing Reading":PercentPassReading,
                                 "Overall Passing Rate":TotalPassCount})            

# Data munging
SchoolSummary = SchoolSummary[["School Type","Total Students","Total School Budget","Per Student Budget",
                                "Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                                 "Overall Passing Rate"]]

SchoolSummary["Total School Budget"] = SchoolSummary["Total School Budget"].map("${:,.2f}".format)
SchoolSummary["Per Student Budget"] = SchoolSummary["Per Student Budget"].map("${:,.2f}".format)

SchoolSummary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,23816
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,23816
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,23816
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,23816
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,23816
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,23816
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,23816
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,23816
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,23816
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,23816


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
# Top 4 performing 
TopSchools = SchoolSummary.sort_values(["Overall Passing Rate"], ascending = False)
TopSchools.head(4)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,23816
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,23816
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,23816
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,23816


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
# Bottom 4 
BottomSchools = SchoolSummary.sort_values(["Overall Passing Rate"], ascending = True)
BottomSchools.head(4)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,23816
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,23816
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,23816
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,23816


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# Calculate math score by grade
NinethGrade =  MergedData[MergedData["Grade"] == "9th"].groupby("School_name").mean()["Math_Score"]
TenthGrade =  MergedData[MergedData["Grade"] == "10th"].groupby("School_name").mean()["Math_Score"]
EleventhGrade =  MergedData[MergedData["Grade"] == "11th"].groupby("School_name").mean()["Math_Score"]
TwelvethGrade =  MergedData[MergedData["Grade"] == "12th"].groupby("School_name").mean()["Math_Score"]

# Math score by grade in table form
MathScoreByGrade = pd.DataFrame({"9th":NinethGrade, "10th":TenthGrade,
                               "11th":EleventhGrade,"12th":TwelvethGrade})            

# Data munging
MathScoreByGrade = MathScoreByGrade[["9th", "10th", "11th", "12th"]]
MathScoreByGrade.index.name = None

# Display the data frame
MathScoreByGrade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
# Reading score by grade
NinethGrade =  MergedData[MergedData["Grade"] == "9th"].groupby("School_name").mean()["Reading_Score"]
TenthGrade =  MergedData[MergedData["Grade"] == "10th"].groupby("School_name").mean()["Reading_Score"]
EleventhGrade =  MergedData[MergedData["Grade"] == "11th"].groupby("School_name").mean()["Reading_Score"]
TwelvethGrade =  MergedData[MergedData["Grade"] == "12th"].groupby("School_name").mean()["Reading_Score"]

# Reading score by grade in table form
ReadingScoreByGrade = pd.DataFrame({"9th":NinethGrade, "10th":TenthGrade,
                               "11th":EleventhGrade,"12th":TwelvethGrade})            

# Data munging
ReadingScoreByGrade = ReadingScoreByGrade[["9th", "10th", "11th", "12th"]]
ReadingScoreByGrade.index.name = None

# Display the data frame
ReadingScoreByGrade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# Scores by school spending
SpendingBins = [0, 585, 615, 645, 675]
SpendingRanges = ["<585", "585-615", "615-645", "645-675"]

# Categorize spending based on spending bins
SchoolSummary["Spending Ranges (Per Student)"] = pd.cut(StudentBudget, SpendingBins, labels = SpendingRanges)


SpendingMath = SchoolSummary.groupby(["Spending Ranges (Per Student)"]).mean()['Average Math Score']
SpendingReading = SchoolSummary.groupby(["Spending Ranges (Per Student)"]).mean()['Average Reading Score']
SpendingPassMath =  SchoolSummary.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Math']
SpendingPassReading =  SchoolSummary.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Reading']
TotalPassingRate =  (SpendingMath + SpendingReading) / 2


# Scores by School Spending in table form
SpendingScore = pd.DataFrame({"Average Math Score":SpendingMath, "Average Reading Score":SpendingReading,
                               "% Passing Math":SpendingPassMath,"% Passing Reading":SpendingPassReading,
                                    "Overall Passing Rate":TotalPassingRate}) 

# Data munging
SpendingScore = SpendingScore[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","Overall Passing Rate"]]

# Display the data frame
SpendingScore




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<585,83.455399,83.933814,90.350436,93.325838,83.694607
585-615,83.599686,83.885211,90.788049,92.410786,83.742449
615-645,79.079225,81.891436,73.021426,83.214343,80.485330
645-675,76.997210,81.027843,63.972368,78.427809,79.012526


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
# Scores by school size
SizeBins = [0, 1000, 2000, 5000]
SizeRanges = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
SchoolSummary["School Size"] = pd.cut(SchoolSummary["Total Students"], SizeBins, labels = SizeRanges)

AvgMath = SchoolSummary.groupby(["School Size"]).mean()['Average Math Score']
AvgReading = SchoolSummary.groupby(["School Size"]).mean()['Average Reading Score']
PercentPassMath =  SchoolSummary.groupby(["School Size"]).mean()['% Passing Math']
PercentPassingReading =  SchoolSummary.groupby(["School Size"]).mean()['% Passing Reading']
TotalPassingRate = SchoolSummary.groupby(["School Size"]).mean()['Overall Passing Rate']


# Scores by School Size in table form
SizeScore = pd.DataFrame({"Average Math Score":AvgMath, "Average Reading Score":AvgReading,
                               "% Passing Math":PercentPassMath,"% Passing Reading":PercentPassReading,
                                    "Overall Passing Rate":TotalPassingRate})   

# Data munging
SizeScore = SizeScore[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","Overall Passing Rate"]]

# Display the data frame
SizeScore

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,NaN,NaN,NaN,79.300643,NaN
Cabrera High School,NaN,NaN,NaN,93.864370,NaN
Figueroa High School,NaN,NaN,NaN,78.433367,NaN
Ford High School,NaN,NaN,NaN,77.510040,NaN
Griffin High School,NaN,NaN,NaN,93.392371,NaN
Hernandez High School,NaN,NaN,NaN,78.187702,NaN
Holden High School,NaN,NaN,NaN,92.740047,NaN
Huang High School,NaN,NaN,NaN,78.813850,NaN
Johnson High School,NaN,NaN,NaN,78.281874,NaN
Large (2000-5000),77.746417,81.344493,67.631335,NaN,23816.0


## Scores by School Type

* Perform the same operations as above, based on school type.

In [13]:
AvgMath = SchoolSummary.groupby(["School Type"]).mean()['Average Math Score']
AvgReading = SchoolSummary.groupby(["School Type"]).mean()['Average Reading Score']
PercentPassMath =  SchoolSummary.groupby(["School Type"]).mean()['% Passing Math']
PercentPassingReading =  SchoolSummary.groupby(["School Type"]).mean()['% Passing Reading']
TotalPassingRate = SchoolSummary.groupby(["School Type"]).mean()['Overall Passing Rate']

# Scores by School Type in table form
TypeScore = pd.DataFrame({"Average Math Score":AvgMath, "Average Reading Score":AvgReading,
                               "% Passing Math":PercentPassMath,"% Passing Reading":PercentPassReading,
                                    "Overall Passing Rate":TotalPassingRate})            
# Data munging
TypeScore = TypeScore[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","Overall Passing Rate"]]
# Display the data frame
TypeScore

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,NaN,NaN,NaN,79.300643,NaN
Cabrera High School,NaN,NaN,NaN,93.864370,NaN
Charter,83.473852,83.896421,90.363226,NaN,23816.0
District,76.956733,80.966636,64.302528,NaN,23816.0
Figueroa High School,NaN,NaN,NaN,78.433367,NaN
Ford High School,NaN,NaN,NaN,77.510040,NaN
Griffin High School,NaN,NaN,NaN,93.392371,NaN
Hernandez High School,NaN,NaN,NaN,78.187702,NaN
Holden High School,NaN,NaN,NaN,92.740047,NaN
Huang High School,NaN,NaN,NaN,78.813850,NaN
